<a href="https://colab.research.google.com/github/julian-reed/InnotechMakeathon/blob/main/Team3MentalHealth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Team Innotech's Makeathon Project!

#Step 1: Data Cleaning

In [1]:
#importing necessary packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
#load in the dataset
url = 'https://raw.githubusercontent.com/julian-reed/InnotechMakeathon/main/MentalHealthDataSet.csv'
df = pd.read_csv(url)

In [3]:
#Visualizing the dataset
df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [4]:
#Percentage of data missing in each column
missing_tot = df.isnull().sum()/len(df) * 100
print(missing_tot)

Timestamp                     0.000000
Age                           0.000000
Gender                        0.000000
Country                       0.000000
state                        40.905481
self_employed                 1.429706
family_history                0.000000
treatment                     0.000000
work_interfere               20.969023
no_employees                  0.000000
remote_work                   0.000000
tech_company                  0.000000
benefits                      0.000000
care_options                  0.000000
wellness_program              0.000000
seek_help                     0.000000
anonymity                     0.000000
leave                         0.000000
mental_health_consequence     0.000000
phys_health_consequence       0.000000
coworkers                     0.000000
supervisor                    0.000000
mental_health_interview       0.000000
phys_health_interview         0.000000
mental_vs_physical            0.000000
obs_consequence          

In [5]:
#Removed due to a significant amount of missnig data
df = df.drop(['state'], axis= 1)
#Removed because when the survey was taken shouldn't correlate with 
#overall mental health
df = df.drop(['Timestamp'], axis= 1)
#Removed because we won't ask for additional comments in practice
df = df.drop(['comments'], axis= 1)
df.head()

,Age,Gender,Country,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,...,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,37,Female,United States,NaN,No,Yes,Often,6-25,No,Yes,...,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,44,M,United States,NaN,No,No,Rarely,More than 1000,No,No,...,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No
2,32,Male,Canada,NaN,No,No,Rarely,6-25,No,Yes,...,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No
3,31,Male,United Kingdom,NaN,Yes,Yes,Often,26-100,No,Yes,...,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,31,Male,United States,NaN,No,No,Never,100-500,Yes,Yes,...,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No


###Cleaning 'Gender'

In [6]:
#Make all elements lower case
gender = df['Gender'].str.lower()

#print unique elements
print(df['Gender'].unique())

['Female' 'M' 'Male' 'male' 'female' 'm' 'Male-ish' 'maile' 'Trans-female'
 'Cis Female' 'F' 'something kinda male?' 'Cis Male' 'Woman' 'f' 'Mal'
 'Male (CIS)' 'queer/she/they' 'non-binary' 'Femake' 'woman' 'Make' 'Nah'
 'All' 'Enby' 'fluid' 'Genderqueer' 'Female ' 'Androgyne' 'Agender'
 'cis-female/femme' 'Guy (-ish) ^_^' 'male leaning androgynous' 'Male '
 'Man' 'Trans woman' 'msle' 'Neuter' 'Female (trans)' 'queer'
 'Female (cis)' 'Mail' 'cis male' 'A little about you' 'Malr' 'p' 'femail'
 'Cis Man' 'ostensibly male, unsure what that really means']


In [7]:
#Group responses into gender groups: male, female and other
male_str = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]       
female_str = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail", "trans-female"]
other_str = ["something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]   

#remove other special instances from the dataset as they likely are not serious
special = ['A little about you', 'p']
df = df[~df['Gender'].isin(special)]

In [8]:
#Replace values
for (row, col) in df.iterrows():

    if str.lower(col.Gender) in male_str:
        df['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)

    if str.lower(col.Gender) in female_str:
        df['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)

    if str.lower(col.Gender) in other_str:
        df['Gender'].replace(to_replace=col.Gender, value='other', inplace=True)

#Check that cleaning was successful
print(df['Gender'].unique())

['female' 'male' 'other']


In [9]:
#replace missing age with median value
df['Age'].fillna(df['Age'].median(), inplace = True)
#Replace other falsified values with median
df['Age'] = df['Age'].replace([-29, 329, 99999999999, -1726], df['Age'].median())
print(df['Age'].unique())

[37 44 32 31 33 35 39 42 23 29 36 27 46 41 34 30 40 38 50 24 18 28 26 22
 19 25 45 21 43 56 60 54 55 48 20 57 58 47 62 51 65 49  5 53 61 11 72]


In [10]:
#Since only 1.4% of values are missing, change those to not self-employed, 
#as that is the statistically more likely real value
df['self_employed'] = df['self_employed'].fillna('No')
print(df['self_employed'].unique())

['No' 'Yes']


In [11]:
#Fill NaN values with unknown
df['work_interfere'] = df['work_interfere'].fillna('Unknown')
print(df['work_interfere'].unique())

['Often' 'Rarely' 'Never' 'Sometimes' 'Unknown']


In [12]:
#Restict to only US survey responses due to low sample size in other countries
df_clean = df
df_clean = df_clean[df_clean.get("Country")=="United States"]
df_clean

,Age,Gender,Country,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,...,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,37,female,United States,No,No,Yes,Often,6-25,No,Yes,...,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,44,male,United States,No,No,No,Rarely,More than 1000,No,No,...,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No
4,31,male,United States,No,No,No,Never,100-500,Yes,Yes,...,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No
5,33,male,United States,No,Yes,No,Sometimes,6-25,No,Yes,...,Don't know,Don't know,No,No,Yes,Yes,No,Maybe,Don't know,No
6,35,female,United States,No,Yes,Yes,Sometimes,1-5,Yes,Yes,...,No,Somewhat difficult,Maybe,Maybe,Some of them,No,No,No,Don't know,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,36,male,United States,No,Yes,No,Rarely,More than 1000,No,No,...,Don't know,Somewhat easy,Maybe,Maybe,Some of them,Some of them,No,No,Don't know,No
1255,32,male,United States,No,Yes,Yes,Often,26-100,Yes,Yes,...,Yes,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No
1256,34,male,United States,No,Yes,Yes,Sometimes,More than 1000,No,Yes,...,Don't know,Somewhat difficult,Yes,Yes,No,No,No,No,No,No
1257,46,female,United States,No,No,No,Unknown,100-500,Yes,Yes,...,Don't know,Don't know,Yes,No,No,No,No,No,No,No


#Step 2: Encoding

In [13]:
#encoded version for cleaned dataset
labelDict = {}
df_encode = df_clean
for feature in df_encode:
    le = preprocessing.LabelEncoder()
    le.fit(df_encode[feature])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    df_encode[feature] = le.transform(df_encode[feature])
    # Get labels
    labelKey = 'label_' + feature
    labelValue = [*le_name_mapping]
    labelDict[labelKey] =labelValue
    
for key, value in labelDict.items():     
    print(key, value)

df_encode

label_Age [5, 11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 60, 62, 65, 72]
label_Gender ['female', 'male', 'other']
label_Country ['United States']
label_self_employed ['No', 'Yes']
label_family_history ['No', 'Yes']
label_treatment ['No', 'Yes']
label_work_interfere ['Never', 'Often', 'Rarely', 'Sometimes', 'Unknown']
label_no_employees ['1-5', '100-500', '26-100', '500-1000', '6-25', 'More than 1000']
label_remote_work ['No', 'Yes']
label_tech_company ['No', 'Yes']
label_benefits ["Don't know", 'No', 'Yes']
label_care_options ['No', 'Not sure', 'Yes']
label_wellness_program ["Don't know", 'No', 'Yes']
label_seek_help ["Don't know", 'No', 'Yes']
label_anonymity ["Don't know", 'No', 'Yes']
label_leave ["Don't know", 'Somewhat difficult', 'Somewhat easy', 'Very difficult', 'Very easy']
label_mental_health_consequence ['Maybe', 'No', 'Yes']
label_phys_health_consequence 

/var/folders/4n/mf_7pzs54fqghgh3zntdwgrc0000gn/T/ipykernel_50391/2722245559.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encode[feature] = le.transform(df_encode[feature])


,Age,Gender,Country,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,...,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,21,0,0,0,0,1,1,4,0,1,...,2,2,1,1,1,2,1,0,2,0
1,28,1,0,0,0,0,2,5,0,0,...,0,0,0,1,0,0,1,1,0,0
4,15,1,0,0,0,0,0,1,1,1,...,0,0,1,1,1,2,2,2,0,0
5,17,1,0,0,1,0,3,4,0,1,...,0,0,1,1,2,2,1,0,0,0
6,19,0,0,0,1,1,3,0,1,1,...,1,1,0,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,20,1,0,0,1,0,2,5,0,0,...,0,2,0,0,1,1,1,1,0,0
1255,16,1,0,0,1,1,1,2,1,1,...,2,1,1,1,1,2,1,1,2,0
1256,18,1,0,0,1,1,3,5,0,1,...,0,1,2,2,0,0,1,1,1,0
1257,30,0,0,0,0,0,4,1,1,1,...,0,0,2,1,0,0,1,1,1,0


In [14]:
df_encode.columns
# classification report

Index(['Age', 'Gender', 'Country', 'self_employed', 'family_history',
       'treatment', 'work_interfere', 'no_employees', 'remote_work',
       'tech_company', 'benefits', 'care_options', 'wellness_program',
       'seek_help', 'anonymity', 'leave', 'mental_health_consequence',
       'phys_health_consequence', 'coworkers', 'supervisor',
       'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence'],
      dtype='object')

#Step 3: Model Training

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# all features except the treatment should be our x axis
X = df_encode[['Age', 'Gender', 'Country', 'self_employed', 'family_history', 'work_interfere',
        'no_employees', 'remote_work', 'tech_company', 'benefits',
       'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence']]

# treatment is our target feature hence it's on y-axis
y = df_encode['treatment'] 
X_train, X_test, y_train, y_test = train_test_split(X, y)

clf =  RandomForestClassifier()
clf.fit(X_train, y_train)

y_prediction = clf.predict(X_test)

print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       0.88      0.77      0.82        97
           1       0.79      0.89      0.84        91

    accuracy                           0.83       188
   macro avg       0.83      0.83      0.83       188
weighted avg       0.84      0.83      0.83       188



In [16]:
#ERIN - trying to increase numbers

# all features except the treatment should be our x axis
X = df_encode[['Age', 'Gender', 'Country', 'self_employed', 'family_history', 'work_interfere',
        'no_employees', 'remote_work', 'tech_company', 'benefits',
       'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence']]

# treatment is our target feature hence it's on y-axis
y = df_encode['treatment'] 


train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.25, random_state = 42)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

rf.fit(train_features, train_labels)

predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')



X_train, X_test, y_train, y_test = train_test_split(X, y)
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
print(classification_report(y_test, y_prediction))
# clf =  RandomForestClassifier()
# clf.fit(X_train, y_train)

# y_prediction = clf.predict(X_test)

# print(classification_report(y_test, y_prediction))

Training Features Shape: (562, 23)
Training Labels Shape: (562,)
Testing Features Shape: (188, 23)
Testing Labels Shape: (188,)
Mean Absolute Error: 0.21 degrees.
Accuracy: -inf %.
              precision    recall  f1-score   support

           0       0.53      0.53      0.53        85
           1       0.61      0.61      0.61       103

    accuracy                           0.57       188
   macro avg       0.57      0.57      0.57       188
weighted avg       0.57      0.57      0.57       188



In [18]:
pip install xgboost

     |████████████████████████████████| 1.8 MB 2.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# all features except the treatment should be our x axis
X = df_encode[['Age', 'Gender', 'Country', 'self_employed', 'family_history', 'work_interfere',
        'no_employees', 'remote_work', 'tech_company', 'benefits',
       'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence']]

# treatment is our target feature hence it's on y-axis
y = df_encode['treatment'] 

X_train, X_test, y_train, y_test = train_test_split(X, y)

clf = XGBClassifier(n_estimators = 200)
clf.fit(X_train, y_train)

y_prediction = clf.predict(X_test)

print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       0.78      0.82      0.80        87
           1       0.84      0.80      0.82       101

    accuracy                           0.81       188
   macro avg       0.81      0.81      0.81       188
weighted avg       0.81      0.81      0.81       188



In [20]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
# all features except the treatment should be our x axis
X = df_encode[['Age', 'Gender', 'Country', 'self_employed', 'family_history', 'work_interfere',
       'no_employees', 'remote_work', 'tech_company', 'benefits',
       'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence']]

# treatment is our target feature hence it's on y-axis
y = df_encode['treatment'] 

X_train, X_test, y_train, y_test = train_test_split(X, y)

clf = svm.SVC()
clf.fit(X_train, y_train)

y_prediction = clf.predict(X_test)

print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       0.67      0.48      0.56        81
           1       0.68      0.82      0.74       107

    accuracy                           0.68       188
   macro avg       0.67      0.65      0.65       188
weighted avg       0.67      0.68      0.66       188



In [22]:
import pickle
pickle.dump(clf, open("model.pkl", "wb"))

## Sources Used:

*   [ML for Mental Health by Kairos](https://www.kaggle.com/code/kairosart/machine-learning-for-mental-health-1/notebook) (Data Cleaning)
*   [Mental Health in Tech Survey](https://www.kaggle.com/datasets/osmi/mental-health-in-tech-survey)  (Dataset)
* [Statology - Pandas Missing values](https://www.statology.org/pandas-count-missing-values/) (Missing data calculation)

